In [1]:
# import json

# # prepare dataset in correct format
# dev_dataset = [json.loads(s) for s in open("/root/data/dev.json", "r").read().split('\n')[:-1]]
# input_list_of_dict = []
# for obj in dev_dataset:
#     instruction = obj['nl'].split("concode_field_sep")[0]
#     input = 'concode_field_sep'.join(obj['nl'].split("concode_field_sep")[1:])
#     input = input.replace('concode_elem_sep', '\n')
#     input = input.replace('concode_field_sep', '\n')
#     assert '\n' not in obj['code']
#     input_list_of_dict.append({"instruction": f"Generate java code\n{instruction}",
#                                "input": input,
#                                "output": obj['code']
#                               }
#                              )

# json.dump(input_list_of_dict, open("/root/data/final_dev.json", 'w+'))
# # json.dump('\n'.join([s['output'] for s in input_list_of_dict]), open("/root/data/final_dev.txt", 'w+'))
# open("/root/data/final_dev.txt", 'w+').write('\n'.join([s['output'] for s in input_list_of_dict]))

# open('/root/data/final_answers_jsonl.json', 'w+').\
# write(
#     '\n'.join([json.dumps({"code": obj['output']})
#                for obj in input_list_of_dict
#               ]
#              )
# )
#
# open("/root/data/final_answers_jsonl.json", "r").read().count('\n'),\
# open("/root/data/final_dev.json", "r").read().count('\n'),\
# open("/root/data/final_dev.txt", "r").read().count('\n')

In [2]:
exp_name = "t2c_concode_220428_v33"

In [3]:
!ls /root/data/

answers.json		  reference_corpus.txt	t2c_dev.json
dev.json		  t2c_10000train.json	t2c_test.json
final_answers_jsonl.json  t2c_1000train.json	t2c_train.json
final_dev.json		  t2c_10answers.json
final_dev.txt		  t2c_answers.json


In [4]:
import sys 
sys.path.append("/root/HSE_diploma/")

import pickle
from datetime import datetime


import numpy as np
import json
import pandas as pd
import os

EXPERIMENT_PATH = "/root/experiments/"
ARTIFACTS_PATH = "/root/temperary_results/"

df_experiments = pd.read_csv("all_experiments.csv").set_index("experiment_name_short")

line = df_experiments[df_experiments['exp_name'] == exp_name]
assert(len(line)==1)
line = line.iloc[0]
line = line.to_dict()

In [5]:
# df_experiments

In [6]:
df_experiments.loc[["9", "10", "11", "12", "13"], "default_model"] = ["decapoda-research/llama-7b-hf"]*5

In [7]:
df_experiments[df_experiments['experiment_config.json']==0]

,exp_name,adapter_model.bin,experiment_config.json,adapter_config.json,fn_config,experiment_name,fn_train_dataset,default_model,MICRO_BATCH_SIZE,BATCH_SIZE,...,bleu_batch_size,GRADIENT_ACCUMULATION_STEPS,fn_eval_dataset,log_bleu_steps_factor,eval_steps,evaluation_strategy,load_best_model_at_end,epoch,num_virtual_tokens,mapping_hidden_dim
experiment_name_short,,,,,,,,,,,,,,,,,,,,,
9,t2c_concode_220428_v9,True,False,True,/root/experiments/experiments/t2c_concode_2204...,NaN,NaN,decapoda-research/llama-7b-hf,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,t2c_concode_220428_v10,True,False,True,/root/experiments/experiments/t2c_concode_2204...,NaN,NaN,decapoda-research/llama-7b-hf,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,t2c_concode_220428_v11,True,False,True,/root/experiments/experiments/t2c_concode_2204...,NaN,NaN,decapoda-research/llama-7b-hf,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,t2c_concode_220428_v12,True,False,True,/root/experiments/experiments/t2c_concode_2204...,NaN,NaN,decapoda-research/llama-7b-hf,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,t2c_concode_220428_v13,True,False,True,/root/experiments/experiments/t2c_concode_2204...,NaN,NaN,decapoda-research/llama-7b-hf,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# df_experiments[df_experiments[]]
# df_experiments['default_model']

In [9]:
# df_experiments
# os.system()

In [10]:
experiment_name = line["experiment_name"]

In [11]:
# Loading files to remote host if they are not exist
if not os.path.exists(os.path.join(EXPERIMENT_PATH, experiment_name)):
    print("loading files")
    #!mkdir {os.path.join(EXPERIMENT_PATH, experiment_name)}
    os.system(f"mkdir {os.path.join(EXPERIMENT_PATH, experiment_name)}")
    for filename in ['adapter_model.bin', 'experiment_config.json', 'adapter_config.json']:
        file_from = f"/root/experiments/experiments/{experiment_name}/{filename}"
        file_to   = f"/root/experiments/{experiment_name}/{filename}"
        print(file_from, '->', file_to)
        #!scp -i ~/.ssh/master_hetzner root@65.108.123.219:{file_from} {file_to} 
        os.system(f"scp -i ~/.ssh/master_hetzner root@65.108.123.219:{file_from} {file_to} ")
else:
    print("File already here")

File already here


In [12]:
!ls -lah /root/experiments/{exp_name}/

total 33M
drwxr-xr-x  2 root root  88 May 26 16:00 .
drwxr-xr-x 10 root root 279 May 26 20:52 ..
-rw-r--r--  1 root root 359 May 26 21:58 adapter_config.json
-rw-r--r--  1 root root 33M May 26 21:58 adapter_model.bin
-rw-r--r--  1 root root 815 May 26 21:58 experiment_config.json


In [13]:
# experiment_name = "/root/experiments/t2c_concode_220428_v38_plustwoepoch_test_copy/"
experiment_config = None
default_model = line['default_model']

if line['experiment_config.json']:
    experiment_config = json.load(open(f"{EXPERIMENT_PATH}/{experiment_name}/experiment_config.json", 
                                           "r"
                                          )
                                     )
    
    default_model = json.load(open(f"{EXPERIMENT_PATH}/{experiment_name}/experiment_config.json", 
                                   "r"
                                  )
                             )['default_model']

# params_iteration = {"temperature": [1.0],
#                     "max_new_tokens": [300]#None, 20, 30, 35, 40, 50, 60, 70, 80, 90, 100] + [45, 47, 49, 51, 53, 55]
#                    }

print("default_model=", default_model)
# print("params_iteration=", params_iteration)

default_model= yahma/llama-7b-hf


In [14]:
from transformers import LlamaForCausalLM as LLaMAForCausalLM
from transformers import LlamaTokenizer as LLaMATokenizer

from peft import PeftModel
import torch


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)


In [15]:
model = LLaMAForCausalLM.from_pretrained(
        pretrained_model_name_or_path = default_model,
        load_in_8bit=True,
        device_map="auto"
    )

model = PeftModel.from_pretrained(
        model = model,
        model_id = os.path.join(EXPERIMENT_PATH, experiment_name),
        torch_dtype=torch.float16,
    )

tokenizer = LLaMATokenizer.from_pretrained(pretrained_model_name_or_path = default_model)

model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
tokenizer.padding_side = "left"
# not sure how necessary this part is, not sure if tloen/alpaca-lora-7b was even trained with EOS and BOS tokens
model.config.bos_token_id = 1
model.config.eos_token_id = 2

model.eval()

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear8bitLt(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          

In [16]:
def verbose_model_tokenizer(model, tokenizer):
    print("tokenizer.pad_token_id", tokenizer.pad_token_id)
    print("tokenizer.eos_token_id", tokenizer.eos_token_id)
    print("tokenizer.bos_token_id", tokenizer.bos_token_id)
    print("tokenizer.eos_token_id", tokenizer.eos_token_id)

    print("model.config.pad_token_id", model.config.pad_token_id)
    print("model.config.eos_token_id", model.config.eos_token_id)
    print("model.config.bos_token_id", model.config.bos_token_id)
    print("model.config.eos_token_id", model.config.eos_token_id)
    if hasattr(model, "print_trainable_parameters"):
        model.print_trainable_parameters()
    
verbose_model_tokenizer(model, tokenizer)

# tokenizer.pad_token_id 0
# tokenizer.eos_token_id 2
# tokenizer.bos_token_id 1
# tokenizer.eos_token_id 2
# model.config.pad_token_id 0
# model.config.eos_token_id 2
# model.config.bos_token_id 1
# model.config.eos_token_id 2
# trainable params: 0 || all params: 6746804224 || trainable%: 0.0


tokenizer.pad_token_id 0
tokenizer.eos_token_id 2
tokenizer.bos_token_id 1
tokenizer.eos_token_id 2
model.config.pad_token_id 0
model.config.eos_token_id 2
model.config.bos_token_id 1
model.config.eos_token_id 2
trainable params: 0 || all params: 6746804224 || trainable%: 0.0


In [17]:
from prompter import generate_test_prompt, get_response
from transformers import GenerationConfig
import math
import tqdm

/root/HSE_diploma/prompter/templates/


In [24]:
generation_config_dict = {"temperature": 1.0,
                          #"penalty_alpha": 2,
                          "max_new_tokens": 500
                         }

generation_config = GenerationConfig(**generation_config_dict)

fn_test_data = "/root/data/final_dev.json"
# fn_test_data = "../data/t2c_answers.json"
# fn_etalon = "/root/data/answers.json"
batch_size = 10
verbose = False

assert model.training == False

lst = json.load(open(fn_test_data, 'rb'))
inputs = lst

prompts = [generate_test_prompt(inp) for inp in inputs]
prompts = np.array(prompts)


In [25]:
lst[0]

{'instruction': 'Generate java code\nactually walks the bag to make sure the count is correct and resets the running total ',
 'input': ' Object _current \n int _total \n DefaultMapBag _parent \n Map _map \n int _mods \n Iterator _support \n boolean add \n boolean add \n Object next \n boolean containsAll \n boolean containsAll \n void clear \n boolean isEmpty \n boolean hasNext \n void remove \n boolean remove \n boolean remove \n Map getMap \n int modCount \n boolean contains \n Iterator iterator \n boolean removeAll \n int size \n boolean addAll \n int hashCode \n boolean equals \n Object[] toArray \n Object[] toArray \n Set uniqueSet \n void setMap \n String toString \n int getCount \n List extractList \n boolean retainAll \n boolean retainAll',
 'output': 'int function ( ) { _total = extractList ( ) . size ( ) ; return _total ; }'}

In [28]:
prompts[0]

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGenerate java code\nactually walks the bag to make sure the count is correct and resets the running total \n\n### Input:\n Object _current \n int _total \n DefaultMapBag _parent \n Map _map \n int _mods \n Iterator _support \n boolean add \n boolean add \n Object next \n boolean containsAll \n boolean containsAll \n void clear \n boolean isEmpty \n boolean hasNext \n void remove \n boolean remove \n boolean remove \n Map getMap \n int modCount \n boolean contains \n Iterator iterator \n boolean removeAll \n int size \n boolean addAll \n int hashCode \n boolean equals \n Object[] toArray \n Object[] toArray \n Set uniqueSet \n void setMap \n String toString \n int getCount \n List extractList \n boolean retainAll \n boolean retainAll\n\n### Response:\n'

In [ ]:
res_list = []
n = math.ceil(len(prompts)/batch_size)

for ind in tqdm.tqdm(range(n)):
    current_prompts = prompts[ind*batch_size: (ind+1)*batch_size]
    if verbose:
        print(ind * batch_size, (ind+1)*batch_size, len(current_prompts))

    tokenized_inputs = tokenizer(list(current_prompts), 
                                 padding=True, 
                                 truncation=True, 
                                 return_tensors="pt"
                                ).to('cuda')



    with torch.no_grad():
        full_output = model.generate(
            **tokenized_inputs,
            generation_config=generation_config
        )

    res_list.extend(tokenizer.batch_decode(full_output, 
                                           skip_special_tokens=False
                                          )
                   )

 24%|██████████▎                               | 49/200 [16:23<33:03, 13.14s/it]

In [ ]:
res_dict = {"real_config": experiment_config,
            "meta_config": line,
            "generation_config": generation_config_dict,
            "fn_test_data": fn_test_data,
#             "fn_etalon": fn_etalon,
            "batch_size": batch_size,
            "verbose": verbose,
            "prompts": list(prompts),
            "res_list": res_list
           }

fn_output = ARTIFACTS_PATH +\
            "predictions" +\
            exp_name+"_"+str(datetime.now()).split('.')[0].replace(' ', '-')
print(fn_output)

In [ ]:
json.dump(res_dict, 
          open(fn_output+'.json', "w+")
         )

pickle.dump(res_dict,
            open(fn_output+'.pickle', "wb")
           )

In [ ]:
def preprocess(s):
    #ToDo rewrite it using Promt Template
    s = get_response(s)
    if "</s>" in s:
        s = s.split('</s>')[0]
    if "### Input" in s:
        s = s.split("### Input")[0]
    try:
        assert "<unk>" not in s
        assert "\n" not in s
        assert "  " not in s
    except Exception as e:
        print("Something wrong with")
        print(s)
        raise e
#     #s = s.split('### Response:\n')[-1]
#     #s = s.replace('\n', '  ')
#     s = s.replace('<unk>', " ")
#     s = ' '.join(s.split(' ')[:100])
#     while '  ' in s:
#         s = s.replace('  ', ' ')

#     if len(s) > 0 and s[0] == ' ':
#         s = s[1:]

#     if self.verbose:
#         print(s)

    return s
# preprocess("\n")

In [ ]:
predictions_list = [preprocess(s) for s in res_list]

In [ ]:
open(fn_output+".txt", 'w+').write('\n'.join(predictions_list))
# 9321

In [ ]:
fn_results = fn_output+".txt"

In [ ]:
fn = fn_output+".txt"
print(fn)

In [ ]:
print(ARTIFACTS_PATH)
!ls {ARTIFACTS_PATH}

In [ ]:
import re
# "/root/temperary_results/predictionst2c_concode_220428_v33_2023-05-26-16:46:49.txt"
def run_evaluation_of_metrics(fn = fn_results):
    #fn_refference = "/root/data/reference_corpus.txt"
    #fn_answers = "/root/data/answers.json"
    
    fn_refference = "/root/data/final_dev.txt"
    fn_answers = "/root/data/final_answers_jsonl.json"
    open("/root/HSE_diploma/load_all_experiments/res.txt", "w+").write('')
    print(f"/root/HSE_diploma/evaluateFromCodeXGlue/;python3 calc_code_bleu.py --refs {fn_refference} --hyp $fn --lang java --params 0.25,0.25,0.25,0.25")
    !cd /root/HSE_diploma/evaluateFromCodeXGlue/;python3 calc_code_bleu.py --refs {fn_refference} --hyp $fn --lang java --params 0.25,0.25,0.25,0.25 > /root/HSE_diploma/load_all_experiments/res.txt
    !cd /root/HSE_diploma/evaluateFromCodeXGlue/;python3 calculate_bleu/evaluator.py -a={fn_answers} -p=$fn  >> /root/HSE_diploma/load_all_experiments/res.txt


def parse_metrics(fn_metrics = "res.txt"):
    s = open('res.txt', "r").read()

    BLEU = re.findall("BLEU: \d+\.\d+", s)
    EM = re.findall("EM: \d+\.\d+", s)
    CodeBLEU = re.findall("CodeBLEU score:  \d+\.\d+", s)

    assert len(BLEU) == len(EM) == len(CodeBLEU) == 1
    BLEU = float(BLEU[0].split(' ')[-1])
    EM = float(EM[0].split(' ')[-1])
    CodeBLEU = float(CodeBLEU[0].split(' ')[-1])

    ngram_match = float(re.findall("ngram match: \d+\.\d+", s)[0].split(' ')[-1]) 
    weighted_ngram_match = float(re.findall("weighted ngram match: \d+\.\d+", s)[0].split(' ')[-1]) 
    syntax_match = float(re.findall("syntax_match: \d+\.\d+", s)[0].split(' ')[-1]) 
    dataflow_match = float(re.findall("dataflow_match: \d+\.\d+", s)[0].split(' ')[-1]) 

    precisions = re.findall("precisions:\s+.+\n", s)[0].split(':  ')[-1][:-1]
    bp = float(re.findall("bp:  \d+\.\d+", s)[0].split(' ')[-1])
    ratio = float(re.findall("ratio:  \d+\.\d+", s)[0].split(' ')[-1])
    translation_length = int(re.findall("translation_length:.*\n", s)[0].split(' ')[-1][:-1])
    reference_length = int(re.findall("reference_length:.*\n", s)[0].split(' ')[-1][:-1])

    # weighted ngram match: 0.2792407600329364, 
    # syntax_match: 0.3445378151260504, 
    # dataflow_match: 0.2908777969018933
    metrics = {"BLEU": BLEU,
               "EM": EM,
               "CodeBLEU": CodeBLEU,
               "ngram_match": ngram_match,
               "weighted_ngram_match": weighted_ngram_match,
               "syntax_match": syntax_match,
               "dataflow_match": dataflow_match,
               "precisions": precisions,
               "bp": bp,
               "ratio": ratio,
               "translation_length": translation_length,
               "reference_length": reference_length,
               "raw": s
              }
    return metrics

run_evaluation_of_metrics(fn = fn_results)
final_metrics = parse_metrics()

final_metrics

/root/HSE_diploma/evaluateFromCodeXGlue/;python3 calc_code_bleu.py --refs /root/data/final_dev.txt --hyp $fn --lang java --params 0.25,0.25,0.25,0.25
INFO:__main__:BLEU: 18.27, EM: 10.0


{'BLEU': 18.27,
 'EM': 10.0,
 'CodeBLEU': 0.2310302252826384,
 'ngram_match': 0.1807804463258241,
 'weighted_ngram_match': 0.23934755329629998,
 'syntax_match': 0.2795031055900621,
 'dataflow_match': 0.22448979591836735,
 'precisions': '[0.875968992248062, 0.6470588235294118, 0.44036697247706424, 0.30303030303030304]',
 'bp': 0.34830125444308524,
 'ratio': 0.4866920152091255,
 'translation_length': 128,
 'reference_length': 263,
 'raw': 'ngram match: 0.1807804463258241, weighted ngram match: 0.23934755329629998, syntax_match: 0.2795031055900621, dataflow_match: 0.22448979591836735\nCodeBLEU score:  0.2310302252826384\nBLEU: 18.27, EM: 10.0\nprecisions:  [0.875968992248062, 0.6470588235294118, 0.44036697247706424, 0.30303030303030304]\nbp:  0.34830125444308524\nratio:  0.4866920152091255\ntranslation_length:  128\nreference_length:  263\n'}

In [153]:
res_dict['final_metrics'] = final_metrics

In [154]:
json.dump(res_dict, 
          open(fn_output+'_final_metrics.json', "w+")
         )

pickle.dump(res_dict,
            open(fn_output+'_final_metrics.pickle', "wb")
           )

In [155]:
fn_output

'/root/temperary_results/predictionst2c_concode_220428_v33_2023-05-26-22:34:18'